# 使用训练模型做预测

In [24]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn
import pickle
import jieba

In [29]:
input_sen = '你人真好'

In [30]:
# 获取停用词
def StopWords():
    with open('train_data/stopwords.txt','r',encoding = 'utf-8') as f:
        lines = f.readlines()
    stopWords = []
    for line in lines:
        stopWords.append(line)
    return stopWords
# 读取训练数据，并去除停用词

In [31]:
# 读取训练数据，并去除停用词
def read_data(input_sen):
    sentence = []
    trans = list(jieba.cut(input_sen.strip(), cut_all=False))

    for word in trans:
        if word not in stopWords:
            sentence.append(word)
    return sentence

In [32]:
stopWords = StopWords()
word2vec_path = 'word2vec/word2vec.pkl' # 词向量地址
file = open(word2vec_path, 'rb')
word2vec_model = pickle.load(file )
vector_size=word2vec_model.vector_size # 词向量的维度
# 将输入数据处理成向量矩阵
MAX_SIZE=25 # 句子的统一长度

In [33]:
def words2Array(line):
    linesArray = []
    wordsArray = []
    steps = []

    t = 0
    p = 0
    for i in range(MAX_SIZE):
        # 如果小于MAX_SIZE 则用0.0的矩阵补齐
        if i < len(line):
            try:
                wordsArray.append(word2vec_model.wv.word_vec(line[i]))
                p = p + 1
            except KeyError:
                    # 如果在词向量模型中没有找到对应词的向量
                t = t + 1
                continue
        else:
            wordsArray.append(np.array([0.0] * vector_size))

    for i in range(t):
        wordsArray.append(np.array([0.0] * vector_size))
    steps.append(p)
    linesArray.append(wordsArray)

    linesArray = np.array(linesArray)
    steps = np.array(steps)
    return linesArray, steps

In [34]:
# 分词和去除停用词
imput_data = read_data(input_sen)

# 构造好输入数据
data, steps = words2Array(imput_data)

In [35]:
num_nodes = 128  #
batch_size = 1 # 一次喂给训练器的数据条数
output_size = 2 # 输出的维度

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.import_meta_graph('model/model-senti.meta')
    saver.restore(sess, tf.train.latest_checkpoint("model/"))
    prediction = tf.get_collection('pred_network')[0]

    graph = tf.get_default_graph()

    input_x = graph.get_operation_by_name('input_x').outputs[0]
    input_steps = graph.get_operation_by_name('steps').outputs[0]

    pre = sess.run(prediction, feed_dict={input_x:data, input_steps: steps})
    print(pre)
    

INFO:tensorflow:Restoring parameters from model/model-senti
[[ 0.28030765  0.71969241]]


In [37]:
print(input_sen)
if pre[0][0] > pre[0][1]:
    print('句子的情感极性：积极')
else:
    print('句子的情感极性：消极')

关系生拉硬拽。朋友们知道火箭助推器吗？就是推动火箭达到预定速度，之后就剥离脱落沙扬娜拉的东西。这片里所有除了男主以外的正面角色，基本都是火箭助推器。皮卡丘自带超越主角的光环不计算在内。但这事儿主要怪男主，牵着不走打着倒退。情感转折突如其来。编剧说他们是好伙伴就是，咱也不知道，咱也不敢问。大难临头分道扬镳，危机散尽相视莫逆。资本主义没有真爱。
句子的情感极性：消极
